<a href="https://colab.research.google.com/github/elangbijak4/LLM-SLM-Examples/blob/main/Rev4_Hitung_Bobot_WQ_WK_WV_k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0

        self.d_model = d_model
        self.num_heads = num_heads
        self.depth = d_model // num_heads

        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)
        self.fc_out = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        return x.view(batch_size, -1, self.num_heads, self.depth).transpose(1, 2)

    def forward(self, x, mask=None):
        batch_size = x.size(0)

        Q = self.wq(x)
        K = self.wk(x)
        V = self.wv(x)

        Q = self.split_heads(Q, batch_size)
        K = self.split_heads(K, batch_size)
        V = self.split_heads(V, batch_size)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.depth)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))

        attention = torch.nn.functional.softmax(scores, dim=-1)
        out = torch.matmul(attention, V).transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        out = self.fc_out(out)
        return out

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff),
            nn.ReLU(),
            nn.Linear(dff, d_model)
        )

        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        attn_output = self.mha(x, mask)
        out1 = self.layernorm1(x + self.dropout1(attn_output))
        ffn_output = self.ffn(out1)
        out2 = self.layernorm2(out1 + self.dropout2(ffn_output))
        return out2

class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, dropout=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_vocab_size, d_model)
        self.pos_encoding = self.positional_encoding(maximum_position_encoding, d_model)
        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, dff, dropout) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        seq_len = x.size(1)
        x = self.embedding(x)
        x *= math.sqrt(self.d_model)
        x += self.pos_encoding[:, :seq_len, :].to(x.device)
        x = self.dropout(x)

        for layer in self.enc_layers:
            x = layer(x, mask)

        return x

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(torch.arange(position).unsqueeze(1), torch.arange(d_model).unsqueeze(0), d_model)
        sines = torch.sin(angle_rads[:, 0::2])
        cosines = torch.cos(angle_rads[:, 1::2])
        pos_encoding = torch.cat([sines, cosines], dim=-1).unsqueeze(0)
        return pos_encoding

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / torch.pow(10000, (2 * (i // 2)) / torch.tensor(d_model, dtype=torch.float32))
        return pos.float() * angle_rates

def create_padding_mask(seq):
    seq = seq == 0
    return seq.unsqueeze(1).unsqueeze(2)

# Parameter Initialization
num_layers = 6
d_model = 512
num_heads = 8
dff = 2048
input_vocab_size = 10000
maximum_position_encoding = 10000

# Instantiate model
encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding)

# Example input
x = torch.randint(0, input_vocab_size, (32, 10))  # batch_size, seq_len

# Create mask
mask = create_padding_mask(x)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(encoder.parameters(), lr=0.001)

# Dummy target for loss calculation
target = torch.randint(0, d_model, (32, 10), dtype=torch.long)  # Random target indices for demonstration

# Training loop
encoder.train()
for epoch in range(100):  # Dummy training loop
    optimizer.zero_grad()
    output = encoder(x, mask)
    output = output.view(-1, d_model)  # Reshape for CrossEntropyLoss
    target = target.view(-1)  # Reshape to match output

    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Print weights for each encoder layer's W_Q, W_K, W_V
    for i, layer in enumerate(encoder.enc_layers):
        print(f"Layer {i+1} Weights after epoch {epoch+1}")
        print("W_Q:", layer.mha.wq.weight.data.shape)
        print(layer.mha.wq.weight.data[:2])  # Print first 2 rows for brevity
        print("W_K:", layer.mha.wk.weight.data.shape)
        print(layer.mha.wk.weight.data[:2])  # Print first 2 rows for brevity
        print("W_V:", layer.mha.wv.weight.data.shape)
        print(layer.mha.wv.weight.data[:2])  # Print first 2 rows for brevity

Output streaming akan dipotong hingga 5000 baris terakhir.
W_Q: torch.Size([512, 512])
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
W_K: torch.Size([512, 512])
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
W_V: torch.Size([512, 512])
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
Layer 2 Weights after epoch 19
W_Q: torch.Size([512, 512])
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
W_K: torch.Size([512, 512])
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
W_V: torch.Size([512, 512])
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
Layer 3 Weights after epoch 19
W_Q: torch.Size([512, 512])
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
W_K: torch.Size([512, 512])
tens